In [5]:
import geopandas as gpd 
import pandas as pd
import matplotlib.pyplot as plt

In [11]:
# load the data
dataset = gpd.read_file('./data.gpkg')
print('dataset size: ', dataset.shape[0])
print('dataset columns: ', dataset.columns.values)

dataset size:  1456
dataset columns:  ['TxID' 'StrNr' 'Strak' 'BdlNr' 'Bandel' 'Forbind_1' 'PlSignFr' 'PlNamnFr'
 'PlSignTi' 'PlNamnTi' 'AntalSpar' 'Elektrif' 'Infraforv' 'UHDist' 'TrSys'
 'TComr' 'Status' 'BdlSeq' 'geometry']


In [12]:
dataset.head()

,TxID,StrNr,Strak,BdlNr,Bandel,Forbind_1,PlSignFr,PlNamnFr,PlSignTi,PlNamnTi,AntalSpar,Elektrif,Infraforv,UHDist,TrSys,TComr,Status,BdlSeq,geometry
0,T230918,1.0,Västra stambanan,612.0,Alingsås-(Partille),A-Bgs,A,Alingsås,Bgs,Bryngenäs,2.0,Ja,Trafikverket infrastruktur,Väst,sysH,G,Öppen,612101.0,"LINESTRING (353981.186 6423216.294, 353980.957..."
1,T231239,13.0,Skånebanan,923.0,(Åstorp)-(Kattarp),Aal-Hlp,Aal,Annedal,Hlp,Hasslarp,1.0,Ja,Trafikverket infrastruktur,Syd,sysH,M,Öppen,923102.0,"LINESTRING (371319.566 6222206.072, 371302.000..."
2,T230282,8.0,Norra stambanan,217.0,Bollnäs-Ljusdal,Ab-Vl,Ab,Arbrå,Vl,Vallsta,1.0,Ja,Trafikverket infrastruktur,Mitt,sysH,Gä,Öppen,217104.0,"LINESTRING (573518.164 6816036.145, 573517.398..."
3,T230178,0.0,Ingår ej i stråkindelning,151.0,Arvidsjaur-(Jörn),Abb-Norl,Abb,Abborrträsk,Norl,Nordlunda,1.0,Nej,Trafikverket infrastruktur,Nord,None,Bdn,Avstängd,151104.0,"LINESTRING (704467.917 7265312.590, 704213.000..."
4,T230400,50.0,(Bollnäs) - Edsbyn - Furudal,251.0,(Bollnäs)-Edsbyn-(Furudal),Aft-Edv,Aft,Alfta,Edv,Edsbyverken,1.0,Nej,Trafikverket infrastruktur,Mitt,None,Gä,Ejunderh,251104.0,"LINESTRING (557316.774 6801745.832, 557177.000..."


In [13]:
df = dataset[['PlSignTi', 'PlSignFr']]
df

,PlSignTi,PlSignFr
0,Bgs,A
1,Hlp,Aal
2,Vl,Ab
3,Norl,Abb
4,Edv,Aft
...,...,...
1451,Kln,Övö
1452,Thn,Öx
1453,Vg,Öx
1454,Hbä,Öä


In [14]:
df = pd.concat([df, pd.DataFrame(data=df[['PlSignTi', 'PlSignFr']], columns=['PlSignFr', 'PlSignTi'])])
df

,PlSignTi,PlSignFr
0,Bgs,A
1,Hlp,Aal
2,Vl,Ab
3,Norl,Abb
4,Edv,Aft
...,...,...
1451,Kln,Övö
1452,Thn,Öx
1453,Vg,Öx
1454,Hbä,Öä


In [15]:
adj_matrix = pd.crosstab(df.PlSignFr, df.PlSignTi)
adj_matrix

PlSignTi,A,Aal,Ab,Abb,Aft,Agb,Agg,Ahm,Aik,Ajr,...,Ös,Öso,Öså,Öte,Öv,Övm,Övn,Övö,Öx,Öä
PlSignFr,,,,,,,,,,,,,,,,,,,,,
A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Aal,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ab,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Abb,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Aft,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Övm,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Övn,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Övö,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
